# Feature Engineering

Will be extended soon. 

## Data Leakage in Train–Test Splits

### What is data leakage?

**Data leakage** occurs when information from the **test set** or from the **future** is used—directly or indirectly—during model training. As a result, model performance appears much better than it truly is and fails to generalize to real‑world data.

> **Rule of thumb:** if the model has access to information during training that would not be available at prediction time, there is data leakage.

---

### Why is data leakage a problem?

The purpose of a train–test split is to simulate unseen future data. When leakage occurs:

- Test data is no longer independent
- Evaluation metrics (accuracy, RMSE, MAPE, …) become overly optimistic
- Model comparison becomes invalid
- Deployed models often perform poorly

---

### Common types of data leakage

#### 1. Leakage through preprocessing

**Wrong approach:**
- Scaling, imputing, or encoding using the **entire dataset** before splitting

**Correct approach:**
- Split the data first
- Fit preprocessing steps **only on training data**
- Apply the learned transformations to both train and test data

✅ Best practice: use machine‑learning **pipelines**.

---

#### 2. Target leakage

Target leakage happens when features contain information that is a direct or indirect consequence of the target variable.

**Example (kijkcijfers):**
- Predicting tomorrow’s viewers
- Using features such as daily ranking, total daily viewers, or advertising revenue

❌ These values are only known after broadcast
✅ They must not be used for prediction

---

#### 3. Time‑based leakage

Common in forecasting problems.

**Wrong:** random train–test split across time

**Correct:** chronological split

- Train: past data
- Test: future data

> Always respect the arrow of time.

---

#### 4. Leakage via feature engineering

Leakage can occur when engineered features use future information.

**Example:**
- Using season‑wide averages to predict early episodes

✅ Use rolling or expanding statistics limited to past data only.

---

#### 5. Leakage via duplication or grouping

Occurs when similar or related observations appear in both train and test sets.

**Example:**
- Different episodes of the same TV show in train and test

✅ Use group‑based splitting (e.g. by program or show).

---

### How to detect data leakage

Ask yourself:

1. Is this feature available at prediction time?
2. Does preprocessing use test‑set statistics?
3. Am I using future information?
4. Are the results suspiciously good?

🚩 Extremely high performance is often a warning sign.

---

### How to prevent data leakage

- Split data **before** preprocessing
- Use pipelines for transformations
- Avoid random splits for time‑dependent data
- Carefully justify every feature
- Document assumptions about data availability

---

### One‑sentence definition

> **Data leakage occurs when information that would not be available at prediction time is used during training or evaluation, leading to overly optimistic performance estimates.**

## Encoding techniques

### Ordinal encoding
see above

### One-hot encoding
see above

### Target encoding
when too manu values for one-hot encoding

### Cycling encoding
for cyclic features, e.g. monthts 1-12, but 1 comes after 12

## Som other feature Engineering techniques
###  Lag features

Pay attention for data leakage!

### Using LLM's to generate extra features

